# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

path = ("../output_data/city_weather.csv")
df = pd.read_csv(path)
df.head()

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed
0,0,leningradskiy,35.28,69.3833,178.4167,93.0,99.0,13.60
1,1,dikson,40.60,73.5069,80.5464,99.0,100.0,1.70
2,2,constantine,90.81,36.3650,6.6147,33.0,75.0,14.97
3,3,butaritari,83.19,3.0707,172.7902,70.0,36.0,9.73
4,4,thompson,48.36,55.7435,-97.8558,93.0,20.0,0.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]
humidity = df["humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
hotel_df = pd.DataFrame(df[(df['temp(f)']<80) & (df['temp(f)']>70) & (df['humidity']<30)])

hotel_df

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed
64,64,medea,72.05,36.2642,2.7539,19.0,80.0,9.04
130,130,gerede,79.25,40.8008,32.1969,23.0,0.0,12.82
204,204,tsumeb,74.44,-19.2333,17.7167,18.0,0.0,15.19
391,391,manyana,71.92,-23.4000,21.7167,17.0,0.0,7.58
577,577,warmbad,78.39,-28.4500,18.7333,12.0,0.0,5.55
578,578,bulawayo,72.64,-20.1500,28.5833,23.0,0.0,6.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ""
#hotel_df['City'] = ""
#hotel_df['Country'] = ""
hotel_df['compound_code'] = ""
hotel_df

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,compound_code
64,64,medea,72.05,36.2642,2.7539,19.0,80.0,9.04,,
130,130,gerede,79.25,40.8008,32.1969,23.0,0.0,12.82,,
204,204,tsumeb,74.44,-19.2333,17.7167,18.0,0.0,15.19,,
391,391,manyana,71.92,-23.4000,21.7167,17.0,0.0,7.58,,
577,577,warmbad,78.39,-28.4500,18.7333,12.0,0.0,5.55,,
578,578,bulawayo,72.64,-20.1500,28.5833,23.0,0.0,6.17,,


In [6]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    #     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "compound_code"] = name_address["results"][0]["plus_code"]["compound_code"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,compound_code
64,64,medea,72.05,36.2642,2.7539,19.0,80.0,9.04,Hôtel M'Sallah,"7Q68+Q6 Médéa, Algeria"
130,130,gerede,79.25,40.8008,32.1969,23.0,0.0,12.82,Hotel Konuk,"Q6W4+29 Gerede/Bolu, Turkey"
204,204,tsumeb,74.44,-19.2333,17.7167,18.0,0.0,15.19,TeaterHuis Guest House,"QP39+RW Tsumeb, Namibia"
391,391,manyana,71.92,-23.4000,21.7167,17.0,0.0,7.58,,
577,577,warmbad,78.39,-28.4500,18.7333,12.0,0.0,5.55,,
578,578,bulawayo,72.64,-20.1500,28.5833,23.0,0.0,6.17,4 On Housman Backpackers,


In [7]:
hotel_df[["compound_code","City","Country"]] = hotel_df["compound_code"].str.split(expand=True)
hotel_df['City'] = hotel_df['City'].str.rstrip(',')

hotel_df

,Unnamed: 0,cities,temp(f),latitude,longitude,humidity,cloudiness,wind_speed,Hotel Name,compound_code,City,Country
64,64,medea,72.05,36.2642,2.7539,19.0,80.0,9.04,Hôtel M'Sallah,7Q68+Q6,Médéa,Algeria
130,130,gerede,79.25,40.8008,32.1969,23.0,0.0,12.82,Hotel Konuk,Q6W4+29,Gerede/Bolu,Turkey
204,204,tsumeb,74.44,-19.2333,17.7167,18.0,0.0,15.19,TeaterHuis Guest House,QP39+RW,Tsumeb,Namibia
391,391,manyana,71.92,-23.4000,21.7167,17.0,0.0,7.58,,None,None,None
577,577,warmbad,78.39,-28.4500,18.7333,12.0,0.0,5.55,,None,None,None
578,578,bulawayo,72.64,-20.1500,28.5833,23.0,0.0,6.17,4 On Housman Backpackers,None,None,None


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [10]:
# Add marker layer ontop of heat map

fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_template)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))